# Introduction

Given a time period of data about what packets are being sent through a conversation between these devices, we attempt to recover and predict the packet loss ratio (how many packets are not received per amount of packets sent) and latency (how long it takes for information to reach the recipient). 

Packet loss and latency are two of the many variables that affect the quality of an internet connection. This model could assist in determining which aspects of the network need improvement and can give insight into developing a better product. 

![alt text](figures/100-500-10000-50000_hist.png)
![alt text](figures/100-500_amp.png)
![alt text](figures/100-500_mean.png)
![alt text](figures/10000-50000_amp.png)
![alt text](figures/10000-50000_mean.png)
![alt text](figures/correlation_matrix.png)
![alt text](figures/label_frequency.png)
![alt text](figures/test_latency_sensitivity_curve.png)
![alt text](figures/test_loss_sensitivity_curve.png)
![alt text](figures/test_residual_latency_plot.png)
![alt text](figures/test_residual_loss_plot.png)